In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import mysql.connector
from tqdm.notebook import tqdm
import pandas as pd
import time

In [2]:
def connect_db():
    db = mysql.connector.connect(
        host="101.101.210.141",
        user="study",
        passwd="study!@#$%",
        database="university_learning"
        # database="aihub_seoul_healthcare"
)
    return db

def mysql_query(database, query:str):
    database.reconnect()
    cur = database.cursor() 
    cur.execute(query)
    result = cur.fetchall()

    return result

In [3]:
db = connect_db()

In [4]:
# eval_data_path = './data/val_unsloth.jsonl'
# eval_data_path = '../data/university/data.jsonl'
# eval_data_path = '../data/welfare/test.jsonl'
eval_data_path = './data/0514university_test_row.jsonl'
evalset = load_dataset("json", data_files = eval_data_path, split='train')
len(evalset)

100

In [5]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "Crysiss/llama-3-8B-sqlcontext-sft-v0.1",
    model_name = "Crysiss/llama-3-8B-university-orpo-v0.1",
    # model_name = "Crysiss/llama-3-8B-sql-orpo-v0.1",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.628 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
prompt_1 = """### Task
Generate a SQL query to answer the following question:
`{}`
 
### Database Schema
This query will run on a database whose schema is represented in this string:
"{}"
An example of the SQL would be 'SELECT * FROM orders WHERE DATE(orderdate) = CURDATE() - INTERVAL 1 DAY AND status = 'COMPLETED'
 
### SQL
Given the database schema, here is the SQL query that answers `{}`:
[SQL]{}"""

In [7]:
test = prompt_1.format(
            f"{evalset[1]['input']}",
            f"{evalset[1]['context']}",
            f"{evalset[1]['input']}",
            "", 
        )

In [8]:
print(test)

### Task
Generate a SQL query to answer the following question:
`각 수업에서 특정 활동 유형이 학생의 평균 성적에 미치는 영향`
 
### Database Schema
This query will run on a database whose schema is represented in this string:
"CREATE TABLE studentvle (code_module varchar(45) NOT NULL -- 'an identification code for a module.', code_presentation varchar(45) NOT NULL -- 'the identification code of the module presentation.', id_student int -- 'a unique identification number for the student.', id_site int -- 'an identification number for the VLE material.', date int -- 'the date of student’s interaction with the material measured as the number of days since the start of the - module-presentation.', sum_click int -- 'the number of times a student interacts with the material in that day.', KEY id_student (id_site), KEY id_site (id_student), KEY code_module (code_module), KEY code_presentation (code_presentation)) ; CREATE TABLE studentassessment (id_assessment int -- 'a unique identification number for the student.',

In [11]:
excute_result = []
query_cnt_result = []
include_query_result = []
cnt_query_result = []
inference_query = []
fail_check = []
FastLanguageModel.for_inference(model) 

In [12]:
for i in tqdm(range(len(evalset))):
    inputs = tokenizer(
    [
        prompt_1.format(
            f"{evalset[i]['input']}",
            f"{evalset[i]['context']}",
            f"{evalset[i]['input']}",
            "", 
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
    result = tokenizer.batch_decode(outputs)
    result = result[0].split("[SQL]")[-1].split("[/SQL]")[0].split("<|end_of_text|>")[0]
    torch.cuda.empty_cache()
    inference_query.append(result)
    # print(f"index:{i}\ninference:{result}")
    
    try:
        infer_query_result = mysql_query(db, result)
        real_query_result = mysql_query(db, evalset['accept'][i])
        comp = any(item in real_query_result for item in infer_query_result)
        excute_result.append(1)
        cnt_query_result.append(len(infer_query_result))
        if len(infer_query_result) == len(real_query_result):
            query_cnt_result.append(1)
        else:
            query_cnt_result.append(0)
            fail_check.append(i)
             
        if comp == True:
            include_query_result.append(1)
        else:
            include_query_result.append(0)
    except Exception as e:
        excute_result.append(0)
        cnt_query_result.append(0)
        query_cnt_result.append(0)
        print(i, e)
        fail_check.append(i)

    time.sleep(0.2)

  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


10 1054 (42S22): Unknown column 'sr.code_module' in 'field list'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


14 1111 (HY000): Invalid use of group function


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


86 1054 (42S22): Unknown column 'age' in 'field list'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


87 1054 (42S22): Unknown column 'si.date_registration' in 'field list'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [11]:
len(inference_query)

100

In [12]:
evalset['accept'][i]

'SELECT c.code_module, c.code_presentation, si.age_band, v.activity_type, AVG(sa.score) AS average_score FROM studentassessment sa JOIN studentinfo si ON sa.id_student = si.id_student JOIN courses c ON si.code_module = c.code_module AND si.code_presentation = c.code_presentation JOIN studentvle sv ON sa.id_student = sv.id_student JOIN vle v ON sv.id_site = v.id_site GROUP BY c.code_module, c.code_presentation, si.age_band, v.activity_type ORDER BY c.code_module, c.code_presentation, si.age_band, v.activity_type;'

In [13]:
for idx, query in enumerate(tqdm(inference_query)):
    try:
            infer_query_result = mysql_query(db, query)
            real_query_result = mysql_query(db, evalset['accept'][idx])
            comp = any(item in real_query_result for item in infer_query_result)
            excute_result.append(1)
            cnt_query_result.append(len(infer_query_result))
            if len(infer_query_result) == len(real_query_result):
                query_cnt_result.append(1)
            else:
                query_cnt_result.append(0)
                fail_check.append(idx)
                
            if comp == True:
                include_query_result.append(1)
            else:
                include_query_result.append(0)
    except Exception as e:
        excute_result.append(0)
        cnt_query_result.append(0)
        query_cnt_result.append(0)
        print(idx, e)
        fail_check.append(idx)

    time.sleep(0.2)

  0%|          | 0/100 [00:00<?, ?it/s]

10 1054 (42S22): Unknown column 'sr.code_module' in 'field list'
14 1111 (HY000): Invalid use of group function
28 1054 (42S22): Unknown column 'si.score_analysis' in 'field list'
43 1054 (42S22): Unknown column 'code_gender' in 'field list'
44 1054 (42S22): Unknown column 'studentinfo.code_site' in 'field list'
45 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'WITH shorter_duration FROM courses WHERE module_presentation_length < shorter_du' at line 1
47 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '/ sa.total_students) * 100 AS vle_percentage FROM studentregistration sr LEFT JO' at line 1
62 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FULL JOIN studentvle sv ON studentregistration

In [13]:
print(f"쿼리 실행 확률 : {round(excute_result.count(1) / len(evalset), 2)}\n\
쿼리 결과 수 같을 확률 : {round(query_cnt_result.count(1) / len(evalset), 2)}")

쿼리 실행 확률 : 0.9
쿼리 결과 수 같을 확률 : 0.66


In [13]:
check_fail_idx = []
for idx, val in enumerate(query_cnt_result):
    if val == 0:
        check_fail_idx.append(idx)

In [15]:
len(check_fail_idx)

358

In [15]:
len(inference_query)

507

In [13]:
import pandas as pd
input = pd.Series(evalset['input'])
rejected = pd.Series(inference_query)
chosen = pd.Series(evalset['output'])
context =  pd.Series(evalset['context'])

df = pd.concat([input, chosen, rejected, context], axis=1)
df.rename(columns={ 0: 'input', 1: 'accept', 2: 'reject', 3: 'context'}, inplace = True)
df.to_csv("../dpo/data/dpo_kor_data.csv")
df

,input,accept,reject,context
0,"CCC수업을 듣는 학생의 성별, 나이, 지역을 알고싶어","SELECT gender, age_band, region FROM studentin...","SELECT s.gender, s.region, s.age_band FROM stu...",CREATE TABLE studentinfo (code_module varchar(...
1,최종 결과가 pass인 학생이 가장 많은 지역,SELECT region FROM studentinfo WHERE final_res...,"SELECT region, COUNT(*) as num_of_students FRO...",CREATE TABLE studentinfo (code_module varchar(...
2,최종 결과가 pass인 학생의 비율이 가장 높은 지역,SELECT region FROM studentinfo GROUP BY region...,"SELECT region, COUNT(*) * 100.0 / (SELECT COUN...",CREATE TABLE studentinfo (code_module varchar(...
3,"연령대 별로 어떤 수업을 많이 듣는지 연령대, 수업코드, 학생 수로 알고싶어","SELECT age_band, code_module, COUNT(id_student...","SELECT age_band, code_module, COUNT(DISTINCT i...",CREATE TABLE studentinfo (code_module varchar(...
4,학생 별 수업 평가결과가 50이상인 학생 수,SELECT COUNT(DISTINCT id_student) AS num_stude...,"SELECT id_student, COUNT(*) as num_assessments...",CREATE TABLE studentassessment (id_assessment ...
...,...,...,...,...
493,"date가 100 이상인 학생들의 평균 성적, 활동 유형 및 이수 학점","SELECT si.id_student, AVG(sa.score) AS average...","SELECT studentassessment.id_student, studentas...",CREATE TABLE studentassessment (id_assessment ...
494,"성별, 지역, 최고 학력, 최종 결과 및 활동 유형에 따른 학생들의 평균 점수와 활동 수","SELECT si.gender, si.region, si.highest_educat...","SELECT studentinfo.gender, studentinfo.region,...",CREATE TABLE studentassessment (id_assessment ...
495,"""AAA"" 모듈에서 각 평가 유형과 평가 유형별로 상위 10% 최고 학력 수준과 하...",WITH education_credits AS (SELECT sa.id_studen...,"SELECT assessment_type, AVG(score) AS avg_scor...",CREATE TABLE studentassessment (id_assessment ...
496,"""BBB"" 모듈에서 각 평가 유형과 평가 유형별로 상위 40% 연령대와 하위 40%...","WITH age_attempts AS ( SELECT sa.id_student, a...","SELECT a.assessment_type, AVG(s.num_of_prev_at...",CREATE TABLE studentassessment (id_assessment ...


In [9]:
i = 1
inputs = tokenizer(
[
    prompt_1.format(
        f"{evalset[i]['input']}",
        f"{evalset[i]['context']}",
        f"{evalset[i]['input']}",
        "", 
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
result = tokenizer.batch_decode(outputs)
result = result[0].split("[SQL]")[-1].split("[/SQL]")[0]

# infer_query_result = mysql_query(db, result)
# real_query_result = mysql_query(db, evalset['output'][i])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
from pprint import pprint
print(result)

SELECT c.code_module, AVG(c.module_presentation_length) as avg_presentation_length, v.activity_type, MAX(sr.date_registration) as max_date_registration FROM courses c JOIN vle v ON c.code_module = v.code_module AND c.code_presentation = v.code_presentation JOIN studentregistration sr ON c.code_module = sr.code_module AND c.code_presentation = sr.code_presentation GROUP BY c.code_module, v.activity_type;


In [13]:
evalset[i]['output']

'SELECT c.code_module, AVG(c.module_presentation_length) AS avg_presentation_length, v.activity_type, AVG(v.week_to - v.week_from) AS avg_activity_duration FROM courses c JOIN vle v ON c.code_module = v.code_module AND c.code_presentation = v.code_presentation JOIN studentRegistration r ON c.code_module = r.code_module AND c.code_presentation = r.code_presentation GROUP BY c.code_module, v.activity_type;'

### test

In [8]:
class EosTokenRewardLogitsProcessor:
  def __init__(self, eos_token_id: int, max_length: int):
    # `eos_token_id`와 `max_length`의 유효성을 검사하는 조건문입니다.
    if not isinstance(eos_token_id, int) or eos_token_id < 0:
        raise ValueError(f"`eos_token_id` has to be a positive integer, but is {eos_token_id}")

    if not isinstance(max_length, int) or max_length < 1:
        raise ValueError(f"`max_length` has to be an integer bigger than 1, but is {max_length}")

    # 생성자 메서드에서 `eos_token_id`와 `max_length`를 클래스 멤버 변수로 저장합니다.
    self.eos_token_id = eos_token_id
    self.max_length = max_length

  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
    cur_len = input_ids.shape[-1]
    # 보상을 증가시키는 과정을 설명하는 주석입니다.
    # eos_token의 보상을 문장 길이에 따라 조절합니다.
    for cur_len in (max(0, int(self.max_length * 0.8)), self.max_length):
      ratio = cur_len / self.max_length
      num_tokens = scores.shape[1]  # 어휘 크기 (단어 집합의 크기)

      # eos_token이 아닌 다른 토큰의 점수를 조정합니다.
      # 선택된 토큰의 부호(sign)의 지수값을 취한 것으로, 음수 값은 0에 가깝게, 양수 값은 1에 가깝게 변환합니다.
      scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]] = \
        scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]] * ratio * 10 * torch.exp(-torch.sign(scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]]))

      # eos_token의 보상을 특정 상수로 설정합니다.
      scores[:, self.eos_token_id] = 1e2 * ratio

    # 조정된 점수가 저장된 scores를 반환합니다.
    return scores

In [10]:
from pprint import pprint
from googletrans import Translator

translator = Translator()

In [20]:
j = 10
korean_input = df['질문'][j]
label = df['답변'][j]
question = translator.translate(korean_input, dest='en').text
inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{question}", # input
            "", 
        )
    ], return_tensors = "pt").to("cuda")
# pprint(inputs)
outputs = model.generate(**inputs,
                         max_new_tokens = 300,
                        #  pad_token_id= tokenizer.eos_token_id,
                        #  eos_token_id= tokenizer.eos_token_id,
                         output_scores=True,
                        #  logits_processor =[EosTokenRewardLogitsProcessor(eos_token_id=tokenizer.eos_token_id, max_length=300)],
                         use_cache = True)
result = tokenizer.batch_decode(outputs)
result = result[0].split("### Response:")[-1].split(";")[0].split("<|end_of_text|>")[0].replace("\n", "") + ";"
torch.cuda.empty_cache()
# print(f"Inference: {result}\nLabel: {evalset['output'][j]}")
print(f"Inference: {result}\nLabel: {label}")
infer_query_result = mysql_query(db, result)
real_query_result = mysql_query(db, label)
comp = any(item in real_query_result for item in infer_query_result)
pritn(f"\ncomparison: {comp}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Inference: SELECT AVG(studied_credits) FROM studentInfo;
Label: SELECT AVG(studied_credits) as avg_credits
FROM studentInfo;


NameError: name 'mysql_query' is not defined